In [5]:
# 这套东西几乎是必须读取的.

import math
import datetime
import multiprocessing as mp
import tushare as ts
import os
import numpy as np
import pandas as pd
import datetime
p = print
# tushare 的初始化
token = '1212fb28fdc360c768e0bd5fca52afa9c8136809afad540f8cf11bc8'
pro = ts.pro_api(token)
# 获取当前上市公司列表，包括股票代码，注册地，行业，上市时间等数据
basic = pro.stock_basic(list_status = 'L')
# 这个 cell 是必须要载入的
basic

,ts_code,symbol,name,area,industry,market,list_date
0,000001.SZ,000001,平安银行,深圳,银行,主板,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,主板,19910129
2,000004.SZ,000004,国农科技,深圳,互联网,主板,19910114
3,000005.SZ,000005,世纪星源,深圳,环境保护,主板,19901210
4,000006.SZ,000006,深振业A,深圳,区域地产,主板,19920427
...,...,...,...,...,...,...,...
3956,688596.SH,688596,正帆科技,上海,专用机械,科创板,20200820
3957,688598.SH,688598,金博股份,湖南,矿物制品,科创板,20200518
3958,688599.SH,688599,天合光能,江苏,电气设备,科创板,20200610
3959,688600.SH,688600,皖仪科技,安徽,电器仪表,科创板,20200703


# 选择股票有关
## 选择几天横盘, 并且超过均线的股票

如 603466 从20200819 到 20200831 的行情

首先当然是导入数据

In [51]:
# 从 basic 中选出所有在长期横盘的股票.
# 研究从 20200101 开始
study_starttime = '20200101'
#####################################
# 设定一个检测天数:check_len

check_len = 15
# 找一段 ,在这一段里找跌幅绝大多数在 2% 以内, 偶尔有两个会在 3%左右.这个限制用 in2_limit 表示
in2_limit = 0.7
in3_limit = 0.95
#############################
# selectdata_list 用于储存最后选择出来的股票时间段.
selectdata_list = []
#############################
# 融资融券交易明细
# Margin trading details
Margin_df = pro.margin_detail(trade_date='20200831')
Margin_list = Margin_df['ts_code']


#############################################
# 对所有 basic 中的股票进行研究:
# 目前测试阶段每次选出 10 个作为观察的例子, 用 select_len 做标记:
select_len = 10
#############################
#for tscode_onestock in basic['ts_code']:
# 从融资融券中选择看看
for tscode_onestock in Margin_list:
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=[5, 10,20])
    if len(oneselect_stockdata) != 0 :
        ############################
        #计算一个平均pct_chg, 作为判定标准
        mean_pctchg = round(np.mean(abs(oneselect_stockdata['pct_chg'])), 4)
        ###########################
        analysis_data = oneselect_stockdata.iloc[:check_len]
        # 要让研究期限内下跌的次数不能高于 60%
        down_rate = sum(analysis_data['pct_chg'] < 0) / len(analysis_data)
        if down_rate < 0.6:
            up2_now = sum(abs(analysis_data['pct_chg']) <= mean_pctchg)/len(analysis_data)
            if up2_now > in2_limit :
                up3_now = sum(abs(analysis_data['pct_chg']) <= (mean_pctchg *1.5))/len(analysis_data)
                if up3_now > in3_limit :
                    # 这里是检验,让开盘收盘很接近.
                    close_beyond_open = []
                    for i in range(len(analysis_data)):
                        temp_data = (analysis_data.iloc[i,2] - analysis_data.iloc[i,5])/analysis_data.iloc[i,5]
                        close_beyond_open.append(round(temp_data,5))
                    up5_sum = sum(abs(np.array(close_beyond_open)) > (mean_pctchg * 0.01))
                    if up5_sum == 0 :
                        # 让最近 3 天(待定) 的5 日均线在最高价和最低价之间.
                        near1 = analysis_data.iloc[0]
                        # 最后一天 单纯超过也算过关 
                        if (near1['high'] > near1['ma5'] and near1['low'] < near1['ma5']) or (near1['high']  > near1['ma5']):
                            near2 = analysis_data.iloc[1]
                            if near2['high'] > near2['ma5'] and near2['low'] < near2['ma5'] :
                                near3 = analysis_data.iloc[2]
                                if near3['high'] > near3['ma5'] and near3['low'] < near3['ma5'] :
                                    ######### 将符合上诉所有要求的股票的 ts_code 放进 list.
                                    selectdata_list.append(analysis_data['ts_code'].iloc[0])
                    
    if len(selectdata_list) > select_len :
        break
                
        
                #break


# 因为读取的数据,只能从前一天开始,而我们要寻找的股票是在当天突破均线的股票.
# 所以在程序里目前只是挑选一直在横盘的股票.
# 突破均线,在之后的高频收集数据的 code 中体现.



#p(oneselect_stockdata[0])

TypeError: object of type 'NoneType' has no len()

In [50]:
# 融资融券交易明细
# Margin trading details
#Margin_df = pro.margin_detail(trade_date='20200831')
#Margin_df['']


,trade_date,ts_code,rzye,rqye,rzmre,rqyl,rzche,rqchl,rqmcl,rzrqye
0,20200831,000001.SZ,4.076781e+09,8.014255e+08,208107728.0,53144926.0,272449024.0,518900.0,2960300.0,4.878207e+09
1,20200831,000002.SZ,5.817807e+09,1.105997e+09,394381658.0,40557293.0,235892846.0,166698.0,2187176.0,6.923804e+09
2,20200831,000006.SZ,7.041911e+08,2.255670e+06,22572885.0,353000.0,17632367.0,101700.0,100.0,7.064467e+08
3,20200831,000008.SZ,3.610076e+08,1.178606e+07,13648894.0,3864280.0,30254661.0,86800.0,170400.0,3.727936e+08
4,20200831,000009.SZ,1.804865e+09,2.676884e+07,67165575.0,3499195.0,68965073.0,136200.0,118200.0,1.831634e+09
...,...,...,...,...,...,...,...,...,...,...
1877,20200831,688596.SH,8.145839e+07,8.312236e+07,23563127.0,3127252.0,22282035.0,123794.0,69800.0,1.645807e+08
1878,20200831,688598.SH,1.750226e+08,5.019760e+07,7005656.0,485517.0,5417831.0,12925.0,15544.0,2.252202e+08
1879,20200831,688599.SH,2.688857e+08,1.860653e+07,15815254.0,1144313.0,15081908.0,41701.0,104216.0,2.874922e+08
1880,20200831,688600.SH,4.761654e+07,3.935603e+06,2427385.0,154156.0,599630.0,25735.0,10443.0,5.155214e+07


In [49]:
selectdata_list
# 000410 这个有问题,应该被去除.



['000551.SZ',
 '000620.SZ',
 '000661.SZ',
 '000918.SZ',
 '000951.SZ',
 '000977.SZ',
 '002001.SZ',
 '002002.SZ',
 '002029.SZ',
 '002169.SZ',
 '002182.SZ']

In [7]:
basic[basic['ts_code'] == '002352.SZ']

,ts_code,symbol,name,area,industry,market,list_date
800,002352.SZ,002352,顺丰控股,深圳,仓储物流,中小板,20100205


In [13]:
basic[basic['name'] == '风语筑'].iloc[0]['ts_code']

'603466.SH'

In [14]:
# 风语筑 的数据
FYZ_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=[5, 10,20])

p(FYZ_stockdata)

ts_code trade_date    open    high     low   close  pre_close  change  \
0    000635.SZ   20200831  7.7900  7.9200  7.7800  7.8000     7.7700  0.0300   
1    000635.SZ   20200828  7.7100  7.8100  7.6400  7.7700     7.7100  0.0600   
2    000635.SZ   20200827  7.6300  7.7700  7.6100  7.7100     7.6400  0.0700   
3    000635.SZ   20200826  7.7200  7.8100  7.6300  7.6400     7.8200 -0.1800   
4    000635.SZ   20200825  7.8700  7.8800  7.7900  7.8200     7.8800 -0.0600   
..         ...        ...     ...     ...     ...     ...        ...     ...   
156  000635.SZ   20200108  7.8973  7.8973  7.6820  7.6918     7.9071 -0.2153   
157  000635.SZ   20200107  7.7994  7.9168  7.7798  7.9071     7.7994  0.1077   
158  000635.SZ   20200106  7.7309  7.8483  7.6526  7.7994     7.7994  0.0000   
159  000635.SZ   20200103  7.8288  7.9756  7.7113  7.7994     7.8190 -0.0196   
160  000635.SZ   20200102  7.7113  7.8581  7.6722  7.8190     7.6918  0.1272   

     pct_chg       vol     amount     ma5     

In [28]:
# 观察 风语筑 从20200611 到 20200617 的数据
index_20200611 = FYZ_stockdata[FYZ_stockdata['trade_date'] == '20200611'].index[0]
# 从 20200611 到 20200617 这五天,是一个调整期(adjust period), 观察其中的特点.
adjperiod = FYZ_stockdata.iloc[index_20200611 - 4 : index_20200611 +1]
# 最近这个调整期,5 日均线在 最高价和最低价之间.
# 上面这个判定 ,也可以改为 在 : 收盘价 和 开盘价之间... 可以通过实验证实其概率. (这个限制可能太高了... 谨慎考虑)
nearone = adjperiod.iloc[0]
p(nearone['high'] > nearone['ma5'])
p(nearone['low'] < nearone['ma5'] and nearone['high'] > nearone['ma5'])



True
True


### 寻找即时融资融券,又是创业板的股票


In [98]:
# grouth_list 所有创业板股票列表.
grouth_list = basic[basic['market'] == '创业板']
grouth_tscode = list(grouth_list['ts_code'])
#p(grouth_list)
# 所有融资融券股票列表
# 要把 20200901 这个改成用代码生成, 先放在这里
Margin_df = pro.margin_detail(trade_date='20200901')
Margin_tscode = list(Margin_df['ts_code'])
#p(Margin_tscode)
# 即是 融资融券 又是 创业板的股票 有 220 只.
grouth_and_margin = [one_coin for one_coin in Margin_tscode if one_coin in grouth_tscode]
p(grouth_and_margin)

['300001.SZ', '300002.SZ', '300003.SZ', '300006.SZ', '300009.SZ', '300010.SZ', '300012.SZ', '300014.SZ', '300015.SZ', '300017.SZ', '300018.SZ', '300020.SZ', '300024.SZ', '300025.SZ', '300026.SZ', '300027.SZ', '300033.SZ', '300034.SZ', '300036.SZ', '300037.SZ', '300038.SZ', '300039.SZ', '300042.SZ', '300053.SZ', '300054.SZ', '300055.SZ', '300058.SZ', '300059.SZ', '300065.SZ', '300066.SZ', '300068.SZ', '300070.SZ', '300072.SZ', '300073.SZ', '300078.SZ', '300079.SZ', '300085.SZ', '300088.SZ', '300091.SZ', '300098.SZ', '300099.SZ', '300101.SZ', '300110.SZ', '300113.SZ', '300115.SZ', '300118.SZ', '300122.SZ', '300123.SZ', '300124.SZ', '300128.SZ', '300130.SZ', '300133.SZ', '300134.SZ', '300136.SZ', '300139.SZ', '300142.SZ', '300144.SZ', '300145.SZ', '300146.SZ', '300147.SZ', '300152.SZ', '300157.SZ', '300166.SZ', '300168.SZ', '300170.SZ', '300174.SZ', '300177.SZ', '300180.SZ', '300182.SZ', '300183.SZ', '300188.SZ', '300189.SZ', '300191.SZ', '300194.SZ', '300197.SZ', '300199.SZ', '300202.SZ'

In [103]:
# 在 又是融资又是融券中选择
selcet_gandm = []
# 从上面的列表中去除股票
for tscode_onestock in grouth_and_margin :
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=averagechose_list)
    near_one = oneselect_stockdata.iloc[0]
    if near_one['high'] > near_one['ma5'] :


        ################################    
        p(oneselect_stockdata)
        break
        

ts_code trade_date     open     high      low    close  pre_close  \
0    300001.SZ   20200901  20.6800  20.8100  20.0800  20.4900    20.6100   
1    300001.SZ   20200831  21.1100  21.2200  20.6100  20.6100    21.0500   
2    300001.SZ   20200828  20.8500  21.1600  20.6600  21.0500    20.9500   
3    300001.SZ   20200827  20.1700  21.0700  19.9000  20.9500    20.1700   
4    300001.SZ   20200826  20.5000  21.1000  20.0100  20.1700    20.7700   
..         ...        ...      ...      ...      ...      ...        ...   
157  300001.SZ   20200108  19.5159  20.5152  19.2661  20.0555    19.3460   
158  300001.SZ   20200107  19.3360  19.8856  19.0662  19.3460    19.3060   
159  300001.SZ   20200106  18.0869  19.6258  17.8471  19.3060    18.0070   
160  300001.SZ   20200103  17.8271  18.1169  17.5773  18.0070    17.8771   
161  300001.SZ   20200102  17.2375  17.9870  17.2375  17.8771    17.1676   

     change  pct_chg        vol  ...      ma5      ma_v_5    ma10     ma_v_10  \
0   -0.1200  

In [102]:
oneselect_stockdata.iloc[0]

ts_code         300881.SZ
trade_date       20200901
open                 56.1
high                94.96
low                 56.03
close                  78
pre_close           14.17
change              63.83
pct_chg           450.459
vol                183460
amount        1.33602e+06
ma5                   NaN
ma_v_5                NaN
ma10                  NaN
ma_v_10               NaN
ma20                  NaN
ma_v_20               NaN
ma30                  NaN
ma_v_30               NaN
ma60                  NaN
ma_v_60               NaN
Name: 0, dtype: object

选择大跌几天,然后形成一个短小十字星的股票.
如 300573 在 20200827 开始 到 20200831 的行情.

### 选择一个融资融券股票
1. 首先要求形成一个短小十字星.

In [60]:
# 融资融券交易明细
# Margin trading details
study_starttime = '20200101'
averagechose_list = [5, 10, 20, 30, 60]
#########################
Margin_df = pro.margin_detail(trade_date='20200901')
for tscode_onestock in Margin_df['ts_code'] :
    oneselect_stockdata = ts.pro_bar(api = pro, ts_code = tscode_onestock, start_date = study_starttime, adj = 'qfq', ma=averagechose_list)
    p(oneselect_stockdata)
    break



ts_code trade_date     open     high      low    close  pre_close  \
0    000001.SZ   20200901  14.9600  15.2300  14.8800  15.1400    15.0800   
1    000001.SZ   20200831  15.3000  15.6800  14.9900  15.0800    15.1300   
2    000001.SZ   20200828  14.2600  15.1800  14.2600  15.1300    14.4600   
3    000001.SZ   20200827  14.4000  14.4600  14.1100  14.4600    14.3700   
4    000001.SZ   20200826  14.6000  14.6100  14.2800  14.3700    14.6000   
..         ...        ...      ...      ...      ...      ...        ...   
157  000001.SZ   20200108  16.7123  16.7615  16.3486  16.3781    16.8598   
158  000001.SZ   20200107  16.8401  16.9876  16.6632  16.8598    16.7812   
159  000001.SZ   20200106  16.7222  17.0466  16.6239  16.7812    16.8893   
160  000001.SZ   20200103  16.6534  17.0171  16.6337  16.8893    16.5845   
161  000001.SZ   20200102  16.3683  16.6632  16.2700  16.5845    16.1717   

     change  pct_chg         vol  ...      ma5       ma_v_5    ma10  \
0    0.0600   0.3979   

In [74]:

##############################
# 计算每一天内 open 和 close 的距离
open_farf_close = (oneselect_stockdata['open'] - oneselect_stockdata['close'])/oneselect_stockdata['close']
# 一天内距离的平均值:
average_open_farf_close = np.mean(abs(open_farf_close))

######################################
one_analysisdata = oneselect_stockdata.iloc[0]
p(one_analysisdata)

ts_code         000001.SZ
trade_date       20200901
open                14.96
high                15.23
low                 14.88
close               15.14
pre_close           15.08
change               0.06
pct_chg            0.3979
vol                813643
amount        1.22834e+06
ma5                14.836
ma_v_5        1.27639e+06
ma10               14.738
ma_v_10        1.2039e+06
ma20              14.5095
ma_v_20       1.40835e+06
ma30               14.219
ma_v_30       1.46587e+06
ma60              14.0107
ma_v_60       1.57857e+06
Name: 0, dtype: object
